# Train house price predictor

The objective of this notebook is to train a model that predicts housing prices. That model will be used in the Flask app to predict a price for an ad on Daft.ie. The user copy-pastes the ad URL into the browser, the app scrapes the ad from Daft.ie, transforms and enriches the data and returns a predicted price. 

The first part of this notebook explores the data and the second part trains the model. This notebook uses the MLpipeline class created in 'MLpipeline.py'. Some functions in that file will also be used live when predicting a price for a user of the app.

The scraping and enriching has already been done and the data is stored in "data/df_ads_mapdata.csv"

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import numpy as np

from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from MLpipeline import MLpipeline

# This makes sure the that the plotly plots can be seen on github (static)
# Remove this to get the interactive plots
pio.renderers.default = "svg"


## Loading data
Loading the data and making a list of variables to be considered as features for the model.

In [ ]:
df_ads = pd.read_csv('data/df_ads_mapdata.csv')

xlist = ['surface','area','property_type','ber_classification',
         'selling_type','price_type','month','year','bathrooms','beds',
         'dist_to_centre','caferestaurants', 'churches', 'health', 
         'parks', 'platforms', 'pubs','schools', 'shops', 'sports', 
         'stations', 'latitude', 'longitude','parking']
all_vars = xlist
all_vars.append('price')

print(df_ads.info())

print("Number of rows: {}".format(df_ads.shape[0]))


The info above shows the size of the dataset and shows that for some variables we have a considerable amount of missing values: 

- price type
- BER classification (this is an energy rating)
- surface

Missing values will be imputed in the pipeline further on, but we should explore what is going on.


## Cleaning data
When the app pulls an ad from the Daft.ie directly, this data needs to be cleaned as well. Therefore we need to define a function that does this cleaning. This function is defined in 'MLpipeline.py' and is called in this notebook later. Imputation of missing values is part of the preprocessing in the ML pipeline.

Some of the cleaning, like removing observations, is only done for the purposes of training the model and does not need to be done everytime a new ad is pulled from the website. These cleaning actions will be done directly here in the notebook.

The price type seems to only be non-missing if it is different from the norm, e.g. a region or a 'from' price. We can recode the missings to a 'normal' category. This is done in MLpipeline.py. The different prices types will have an influence on the price but because it is a significant number we keep the data. We include the variable in the model, however.
 

In [ ]:
df_ads.price_type.value_counts()

For the BER I cant discover anything unusual. Most likely the missing ratings are for properties of which the owner did not test the energy efficiency. The missing values will be imputed in the ML pipeline.

In [ ]:
df_ads.ber_classification.value_counts()


The property type shows that most of the properties on the website are apartments, (semi-)detached, terraced or end-of-terrace properties. There are also some smaller categories: bungalows, duplexes, sites, studios and houses. Sites will be removed from the data for training the model. The aim is to predict prices for properties, not land. 

In [ ]:
print(df_ads.property_type.value_counts())

df_ads = df_ads.drop(df_ads[df_ads.property_type=='site'].index)

print("Number of rows: {}".format(df_ads.shape[0]))

The selling type could considerably influence the price. Prices for houses that are auctioned will not be comparable to houses that are not auctioned. Only 'private-treaty' is kept and so are missing values which effectively imputes them as 'private-treaty'.  

In [ ]:
print(df_ads.selling_type.value_counts())

df_ads = df_ads[(df_ads!="auction") & (df_ads!='private-tender') & (df_ads!='tender')]
df_ads = df_ads.drop('selling_type',axis=1)

print("Number of rows: {}".format(df_ads.shape[0]))

Checking for outliers in the numerical variables

In [ ]:
df_ads.loc[:,df_ads.columns.isin(all_vars)].describe(percentiles=[.001, .01, .05, .25, .5, .75, .95, .99, 0.999]).applymap(lambda x: format(x, ".1f")).transpose()

There seem to be some extreme cases for the number of beds and bathrooms. Removing properties with more than 10 bedrooms or bathrooms. Also the surface area (square meters) can be rather high in a few cases which is either a mistake or it is properties with a signficant amount of land which is not surface area of the property itself. These will be removed because they would be very atypical.

In [ ]:
df_ads = df_ads[df_ads.bathrooms<=10]
df_ads = df_ads[df_ads.beds<=10]
df_ads = df_ads[(df_ads.surface<=1000) | pd.isnull(df_ads.surface)]

print("Number of rows: {}".format(df_ads.shape[0]))

The distribution of the price is heavily skewed. House prices around 300k are most common but in our dataset it goes up to 10 million euros. 

In [ ]:
px.histogram(x=df_ads.price, nbins=200, marginal="box", labels={"x": "price"})


Some properties are very cheap. The minimum price in the dataset is actually 20k. Less 0.1% of the data has a price of around 125k or lower. Anything under a 100k is probably an atypical property so they will be removed. Similarly, houses priced over 3 million euros will also be removed. Finally, dropping rows with missing prices. The plot below shows the price distribution after cleaning. 

For the estimation its probably better to take the log of the prices which returns a distribution that is much closer to the normal distribution.

In [ ]:
df_ads = df_ads[(df_ads.price>100000) & (df_ads.price<3000000)]       
df_ads = df_ads.dropna(subset=['price'])

print("Number of rows: {}".format(df_ads.shape[0]))

px.histogram(x=df_ads.price, nbins=200, marginal="box", labels={"x": "price"}, log_x=True)

In [ ]:
df_ads = df_ads.reset_index()
print("Number of rows: {}".format(df_ads.shape[0]))

In [ ]:
xlist = ['surface','area','property_type','ber_classification','price_type', 'month','year','bathrooms','beds','dist_to_centre','caferestaurants', 'churches', 'health',       'parks', 'platforms', 'pubs','schools', 'shops', 'sports', 'stations', 'latitude', 'longitude','facility','one_unit']

numeric_features = ['surface','bathrooms', 'beds', 'dist_to_centre', 'caferestaurants', 'churches', 'health', 'parks', 'platforms', 'pubs','schools', 'shops', 'sports', 'stations', 'latitude', 'longitude']

categorical_features = xlist.copy()

for x in numeric_features:
    categorical_features.remove(x)

In [ ]:
mlp_linreg = MLpipeline(df_ads)

parametersGrid = {}

mlp_linreg.set_estimator(LinearRegression(), 
                         parametersGrid, xlist,
                         numeric_features, categorical_features)
mlp_linreg.run()


In [ ]:
mlp_enet = MLpipeline(df_ads)

parametersGrid = {"estimator__alpha": [0.001,0.01, 0.1,0.5,0.9,1],
                  "estimator__l1_ratio": [0.01, 0.1, 0.3, 0.5, 0.7, 0.9]}

mlp_enet.set_estimator(ElasticNet(max_iter=10000), 
                         parametersGrid, xlist, 
                         numeric_features, categorical_features)
mlp_enet.run()


In [ ]:
mlp_rf = MLpipeline(df_ads)

parametersGrid = {'estimator__n_estimators': [2500,3000,3500],
                  'estimator__max_features': ['auto','log2'],
                  'estimator__min_samples_leaf': [1,5,20,50]}

mlp_rf.set_estimator(RandomForestRegressor(random_state=42), 
                     parametersGrid, xlist,
                     numeric_features, categorical_features) 

mlp_rf.run()

In [ ]:
mlp_xgb = MLpipeline(df_ads)

parametersGrid = {'estimator__n_estimators': [2500,3000,3500],
                  'estimator__max_depth': [1,3,5,7,9,11],
                  'estimator__learning_rate': [0.01,0.1,0.3],
                  'estimator__gamma': [0.0, 0.2, 0.4],
                  'estimator__colsample_bytree': [0.1,0.3,0.5]}

mlp_xgb.set_estimator(XGBRegressor(objective='reg:squarederror', random_state=42),
                         parametersGrid, xlist,
                         numeric_features, categorical_features)
mlp_xgb.run()


In [ ]:
df0 = pd.DataFrame({'y_pred': mlp_linreg.y_pred,
                    'y_test': mlp_linreg.y_test,
                    'estimator': 'Regression'})

df1 = pd.DataFrame({'y_pred': mlp_enet.y_pred,
                    'y_test': mlp_enet.y_test,
                    'estimator': 'ElasticNet'})

df2 = pd.DataFrame({'y_pred': mlp_rf.y_pred,
                    'y_test': mlp_rf.y_test,
                    'estimator': 'RandomForest'})
              
df3 = pd.DataFrame({'y_pred': mlp_xgb.y_pred,
                    'y_test': mlp_xgb.y_test,
                    'estimator': 'GradientBoost'})

df = df0.append(df1, ignore_index=True)
df = df.append(df2, ignore_index=True)
df = df.append(df3, ignore_index=True)

fig = px.scatter(df, x="y_test", y="y_pred", color="estimator", 
                marginal_y="box",
                labels={'y_test':'Observed',
                        'y_pred': 'Predicted'})
                        
fig.update_traces(marker=dict(size=6,
                              opacity=0.5))
fig.show()

Model fit seems to be comparable for the different estimators. Generally it seems that overfitting is not an issue, e.g. linear regression works better than elastic net and the best paramters of the gridsearch for elastic net are those very close to linear regression. Gradient stochastic boosting outperforms the others slightly so this model will be used to predict housing prices.

In [ ]:
mlp_xgb.save_model('model/model.pkl')